<h1 id="Visualisation-tools-for-Middlebury-2014-results">Visualisation tools for Middlebury 2014 results</h1>
<p>Admittedly, this dataset proved to be more challenging. Not just because of the higher number of the scenes and their more challenging dimensions, but:</p>
<p>The disparity maps provided were in "pfm" format. This image format allows for 24 bit floating point image representation. However, even using the provided calibration files to each scene, transforming them to integer disparities proved to be challenging. The provided calibration files for each dataset are supposed to give information on the disparity ranges, minimum and maximum values that are encoded in each image.</p>
This was used to scale these disparities to the level the algorithm outputs.
To test the accuracy of converting the floating point disparities to integer values the following was done:</p>
<ol>
    <li>
A scene, namely "Teddy" was converted to integer (8bit) values based on the provided configuration file.</li>
    <li>As this is a scene present in both the 2003 and 2014 datasets, it had integer disparity maps (version 2003) and floating point ones (version 2014).<pre>Please note that here, "integer values" means the values saved in the disparity map file, not actual disparity values, as the 2003 version encoded disparities with 8 bit at a quater pixel accuracy.</pre>
</li>
    <li>The two versions disparity maps were compared.</li>
    <li>Equivalence, apart from a small percentage of pixels could not be established.</li>
    <li>There was an average of 2px difference between the two versions of the same frame.</li>
</ol>        
Please regard this as a "caveat" as using the old framework and old images ("Teddy" scenes) the algorithm scored a 9% bad2 value, whereas here it is significantly, typicall by at least factor of 2 greater. The reason for this is the aforementioned conversion error to which the reason is unknown.</p>
<h2 id="Contents">Contents</h2>
<ol>
<li>Scenes w.r.t. {metric} (selection plot)</li>
<li>Scenes w.r.t. {metric} with color coded "epochs"</li>
<li>"Epochs w.r.t. {metric} with color coded scenes</li>
</ol>

In [1]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import sys
sys.path.append("../../")
import os
import glob
import cv2

import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import HBox, VBox, Button

from components.utils import plotly_helpers as ph

In [2]:
log_directory = os.path.join("..", "logs")
csv_files = glob.glob(os.path.join(log_directory, "*.csv"))
filenames = [os.path.split(csv_file)[-1] for csv_file in csv_files]
opts = [csv_tuple for csv_tuple in zip(filenames, csv_files)]

In [3]:

available_metrix = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']

metrics_selector = widgets.Dropdown(
    options=[(m,m) for m in available_metrix],
    description='Metrics:',
    value="bad2"
)


nonoccluded = widgets.Dropdown(
    options=[("yes", False), ("No", True)],
    description='Nonoccluded:'
)


### Please select metrics and whether occlusions are counted as errors

In [4]:
VBox([metrics_selector, nonoccluded])

In [5]:

selected_file = os.path.join("..","..", "benchmarking", "MiddEval", "custom_log", "all_benchmarking.csv")

df = ph.load_n_clean(selected_file)

##Filtering to selected occlusion parameter

df = df[df["are_occlusions_errors"]==nonoccluded.value]

number_of_samples = df.shape[0]

In [6]:
### Dashboard 1


from ipywidgets import Image, Layout

img_widget = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))

fig_a = ph.get_figure_widget (df, "scene", metrics_selector.value, 
                           "Scene w.r.t."+metrics_selector.value)
fig_b = ph.get_figure_widget (df, "match", "kernel_size", "Kernel sizes w.r.t. match values")


figs = [fig_a, fig_b]
ph.bind_hover_function(figs, img_widget, df)
ph.bind_brush_function(figs, df)

button = ph.get_reset_brush_button(figs)
dashboard1 = VBox([button, fig_a,
                  HBox([img_widget, fig_b])])


### Dashboard 2


traced_fig_1, dfs_1 = ph.get_figure_widget_traced(df, "scene", metrics_selector.value, "experiment_id")

traced_fig_widget_1 = go.FigureWidget(traced_fig_1)



traced_fig_1_imw_1 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
traced_fig_1_imw_2 = Image(value=df["loaded_gts"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))

#figs, img_widget, selected_scene_df
ph.bind_hover_function2([traced_fig_widget_1], traced_fig_1_imw_1, dfs_1, img_widget_groundtruth=traced_fig_1_imw_2)

turn_the_lights_on = ph.get_dropdown_widget(["On", "Off"], label="Turn plots:", values = [True, False])

ph.bind_dropdown_switch_traces_fn(turn_the_lights_on, traced_fig_widget_1)

dashboard2 = VBox([turn_the_lights_on, traced_fig_widget_1, HBox([traced_fig_1_imw_1,traced_fig_1_imw_2])])


### Dashboard 3


traced_fig_2, dfs_2 = ph.get_figure_widget_traced(df, "experiment_id", metrics_selector.value, "scene")

traced_fig_widget_2 = go.FigureWidget(traced_fig_2)

traced_fig_2_imw_1 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
traced_fig_2_imw_2 = Image(value=df["loaded_gts"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))



#figs, img_widget, selected_scene_df
ph.bind_hover_function2([traced_fig_widget_2], traced_fig_2_imw_1, dfs_2, img_widget_groundtruth=traced_fig_2_imw_2)

turn_the_lights_on_2 = ph.get_dropdown_widget(["On", "Off"], label="Turn plots:", values = [True, False])

ph.bind_dropdown_switch_traces_fn(turn_the_lights_on_2, traced_fig_widget_2)


dashboard3 = VBox([turn_the_lights_on_2, traced_fig_widget_2, HBox([traced_fig_2_imw_1,traced_fig_2_imw_2])])


### Scene w.r.t. {metric} (selection plot)
<ol>
    <li>The following figure allows to use the "lasso" tool as a tool of selection.</li>
    <li>As a result, the relevant datapoints and their corresponding values in the figure in the bottom right corner will be highlighted.</li>
    <li>Pressing the "clear selection" button will reset the figure.</li>
    <li> Additionally, if a data point is hovered, the corresponding disparity output value will be displayed in the bottom right corner.</li>

</ol>

In [7]:
dashboard1

    'data': [{'custo…

### Scenes w.r.t. {metric} with color coded "epochs"
An "epoch" in this context means an experiment with the same settings evaluated across every scene in the Middlebury 2004 training dataset.<br>
<ol>
    <li>The following figure allows to turn all the plots on and off</li>
    <li>Additionally, their visibiliy can also be handled by interacting with their legend entries on the right side of the plot.
    </li>
    <li> Therefore custom comparison can be made between different scenes, kernel sizes and match values. </li>
    <li> TThe figure in the bottom left corner shows the corresponding disparity map. </li>
    <li> TThe figure in the bottom right corner shows the corresponding ground truth disparity map. </li>
</ol>

In [8]:
dashboard2

### "Epoch" w.r.t. {metric} with color coded scenes
An "epoch" in this context means an experiment with the same settings evaluated across every scene in the Middlebury 2004 training dataset.<br>
<ol>
    <li>The following figure allows to turn all the plots on and off</li>
    <li>Additionally, their visibiliy can also be handled by interacting with their legend entries on the right side of the plot.
    </li>
    <li> Therefore custom comparison can be made between different scenes, kernel sizes and match values. </li>
    <li> TThe figure in the bottom left corner shows the corresponding disparity map. </li>
    <li> TThe figure in the bottom right corner shows the corresponding ground truth disparity map. </li>
</ol>

In [9]:
dashboard3